Anomalie w ruchu sieciowym


In [1]:
#Podłączenie do dysku google
#https://www.kaggle.com/datasets/anushonkar/network-anamoly-detection/data?select=Network+Anamoly+Detection.docx
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#import bibliotek
import numpy as np
import pandas as pd

In [3]:
kolumny=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land",
"wrong_fragment","urgent","hot","num_failed_logins","logged_in",
"num_compromised","root_shell","su_attempte","num_root","num_file_creations",
"num_shells","num_access_files","num_outbound_cmds","is_host_login",
"is_guest_login","count","srv_count","serror_rate", "srv_serror_rate",
"rerror_rate","srv_rerror_rate","same_srv_rate", "diff_srv_rate", "srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate",
"dst_host_diff_srv_rate","dst_host_same_src_port_rate",
"dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
"dst_host_rerror_rate","dst_host_srv_rerror_rate","attack_name", "last_flag"]

In [4]:
df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/pracownia_problemowa/pliki/Train.csv')
df_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/pracownia_problemowa/pliki/Test.csv', header=None, names=kolumny)

# Połącz DataFrame'y pionowo
df = pd.concat([df_train, df_test], ignore_index=True)
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_name,last_flag
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [5]:
result = df.groupby('attack_name').size()
print(result.to_string())

attack_name
apache2              737
back                1315
buffer_overflow       50
ftp_write             11
guess_passwd        1284
httptunnel           133
imap                  12
ipsweep             3740
land                  25
loadmodule            11
mailbomb             293
mscan                996
multihop              25
named                 17
neptune            45871
nmap                1566
normal             77054
perl                   5
phf                    6
pod                  242
portsweep           3088
processtable         685
ps                    15
rootkit               23
saint                319
satan               4368
sendmail              14
smurf               3311
snmpgetattack        178
snmpguess            331
spy                    2
sqlattack              2
teardrop             904
udpstorm               2
warezclient          890
warezmaster          964
worm                   2
xlock                  9
xsnoop                 4
xterm        

In [6]:
#Wybieramy 2 ataki o podobnej liczebności

#Ruch, który zostawiamy
wybrane_nazwy_atakow = ['normal', 'neptune']

# Tworzenie warunku logicznego dla wierszy do usunięcia
warunek_do_usuniecia = ~df['attack_name'].isin(wybrane_nazwy_atakow)

# Usunięcie wierszy spełniających warunek
df = df[~warunek_do_usuniecia]

# Wyświetlenie liczby rekordów dla poszczególnych ataków po usunięciu wierszy
result = df.groupby('attack_name').size()
print(result.to_string())


attack_name
neptune    45871
normal     77054


In [7]:
#Zbalansowanie klasy normal
indeksy_do_usuniecia = df[df['attack_name'] == 'normal'].sample(31183).index

#Usuń wybrane wiersze
df = df.drop(indeksy_do_usuniecia)

result = df.groupby('attack_name').size()
print(result.to_string())

attack_name
neptune    45871
normal     45871


In [8]:
# Usuń dane redundatne
df_no_duplicates = df.drop_duplicates()
# Usuń puste rekordy
df_cleaned = df.dropna()

In [9]:
#Funckja licząca entropię dla dalej kolumny
def find_entropy(attribute):
  entropy_node = 0
  values = df[attribute].unique()
  for value in values:
    fraction = df[attribute].value_counts()[value]/len(df[attribute])
    entropy_node += -fraction * np.log2(fraction)
  entropy_node
  return entropy_node

In [10]:
#Liczymy entropię dla każdej kolumny
entropia_dict_list = []
for attribute in kolumny:
    entropia = find_entropy(attribute)
    print(f"{attribute}: entropia = {entropia}")
    entropia_dict_list.append({"kolumna": attribute, "entropia": entropia})

duration: entropia = 0.6778525738036215
protocol_type: entropia = 0.5171061227889209
service: entropia = 3.9768366032269253
flag: entropia = 1.546002612791666
src_bytes: entropia = 5.181851220719943
dst_bytes: entropia = 5.679640081181062
land: entropia = 0.00036903433658853045
wrong_fragment: entropia = 0.00453145909717568
urgent: entropia = 0.0009252563146797323
hot: entropia = 0.10209180545091241
num_failed_logins: entropia = 0.007481807567608743
logged_in: entropia = 0.9410138488144995
num_compromised: entropia = 0.035312817781450116
root_shell: entropia = 0.011327119796920125
su_attempte: entropia = 0.007267991345380221
num_root: entropia = 0.05221364406271347
num_file_creations: entropia = 0.022946433052124654
num_shells: entropia = 0.003229758165766423
num_access_files: entropia = 0.028102432917143993
num_outbound_cmds: entropia = 0.0
is_host_login: entropia = 0.000195417386982292
is_guest_login: entropia = 0.052717957714586525
count: entropia = 7.0536369450185346
srv_count: ent

In [11]:
# Definiuj próg entropii i usuwamy kolumny o słabej entropii (ale nie usuwamy attack_name)
prog_entropii = 3
tmp_list = []
for entropia_dict in entropia_dict_list:
    if(entropia_dict['entropia'] < prog_entropii):
        if(entropia_dict['kolumna'] != 'attack_name'):
          df.drop(entropia_dict['kolumna'], axis=1, inplace=True)
          tmp_list.append({"kolumna": entropia_dict['kolumna'], "entropia": entropia_dict['entropia']})

for item in tmp_list:
      entropia_dict_list.remove(item)

for entropia_dict in entropia_dict_list:
    print(f"{entropia_dict['kolumna']}: entropia= {entropia_dict['entropia']}")

service: entropia= 3.9768366032269253
src_bytes: entropia= 5.181851220719943
dst_bytes: entropia= 5.679640081181062
count: entropia= 7.0536369450185346
srv_count: entropia= 5.054546409607462
same_srv_rate: entropia= 3.4405643651725732
dst_host_count: entropia= 3.431380279098006
dst_host_srv_count: entropia= 5.409263152512489
dst_host_same_srv_rate: entropia= 4.244992701260231
dst_host_diff_srv_rate: entropia= 3.2279244011425243
attack_name: entropia= 1.0


In [12]:
#Sprawdzamy, czy napewno się usunęły
df.head()

,service,src_bytes,dst_bytes,count,srv_count,same_srv_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,attack_name
1,other,146,0,13,1,0.08,255,1,0.00,0.60,normal
2,private,0,0,123,6,0.05,255,26,0.10,0.05,neptune
5,private,0,0,121,19,0.16,255,19,0.07,0.07,neptune
6,private,0,0,166,9,0.05,255,9,0.04,0.05,neptune
7,private,0,0,117,16,0.14,255,15,0.06,0.07,neptune


In [13]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Tworzenie kopii ramki danych
df_normalized = df.copy()

# Stosowanie LabelEncoder dla każdej kolumny zawierającej dane kategoryczne
label_encoder = LabelEncoder()
for column in df_normalized.select_dtypes(include=['object']).columns:
    if column != 'attack_name':
        df_normalized[column] = label_encoder.fit_transform(df_normalized[column])

# Standaryzacja (bez kolumny "attack_name")
scaler = StandardScaler()
columns_to_normalize = df_normalized.columns[df_normalized.columns != 'attack_name']
df_normalized[columns_to_normalize] = scaler.fit_transform(df_normalized[columns_to_normalize])

df = df_normalized

In [14]:
df.columns

Index(['service', 'src_bytes', 'dst_bytes', 'count', 'srv_count',
       'same_srv_rate', 'dst_host_count', 'dst_host_srv_count',
       'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'attack_name'],
      dtype='object')

In [15]:
df

,service,src_bytes,dst_bytes,count,srv_count,same_srv_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,attack_name
1,0.659708,-0.019168,-0.045511,-0.831491,-0.392516,-0.993697,0.622687,-0.925695,-0.990429,5.832749,normal
2,0.921703,-0.019599,-0.045511,0.228946,-0.289743,-1.059269,0.622687,-0.700433,-0.766010,-0.031451,neptune
5,0.921703,-0.019599,-0.045511,0.209666,-0.022534,-0.818838,0.622687,-0.763506,-0.833336,0.181792,neptune
6,0.921703,-0.019599,-0.045511,0.643481,-0.228079,-1.059269,0.622687,-0.853611,-0.900662,-0.031451,neptune
7,0.921703,-0.019599,-0.045511,0.171104,-0.084198,-0.862553,0.622687,-0.799548,-0.855778,0.181792,neptune
...,...,...,...,...,...,...,...,...,...,...,...
148506,-0.519271,-0.018790,-0.012862,-0.947175,-0.392516,1.017179,-1.172995,1.362968,1.253763,-0.564560,normal
148507,0.921703,-0.019599,-0.045511,1.308664,-0.207525,-1.081126,0.622687,-0.844601,-0.900662,0.181792,neptune
148509,-0.322774,-0.019599,-0.045511,0.267508,-0.043089,-0.862553,0.622687,-0.772517,-0.833336,-0.031451,neptune
148512,1.249197,-0.017257,-0.038813,-0.947175,-0.392516,1.017179,-1.084864,0.335773,0.625389,0.075171,normal


In [ ]:
df.to_csv('/content/drive/My Drive/Colab Notebooks/pracownia_problemowa/pliki/attack.csv', index=False)
